In [1]:
# ============================================================================
# FASE 2 - OPTIMIZADO PARA SPARK 3.5.1 + DELTA LAKE 3.0
# ============================================================================

# PASO 0: REINICIAR SPARK CON VERSIONES CORRECTAS
try:
    spark.stop()
except:
    pass

import time
time.sleep(3)

from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.functions import col, lower, regexp_replace, translate, length, trim

from pyspark.ml.feature import (
    Tokenizer, StopWordsRemover, Word2Vec, 
    StringIndexer, OneHotEncoder, VectorAssembler,
    StandardScaler, PCA
)
from pyspark.ml import Pipeline
from pyspark.ml.stat import Correlation
import numpy as np

spark = (
    SparkSession.builder
    .appName("Bronze_to_Silver_Optimized")
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
    .config("spark.jars.packages", 
            "org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.1,"
            "io.delta:delta-spark_2.12:3.0.0")
    .config("spark.driver.memory", "4g")
    .config("spark.executor.memory", "4g")
    .config("spark.driver.maxResultSize", "2g")
    .config("spark.sql.shuffle.partitions", "50")
    .config("spark.sql.adaptive.enabled", "true")
    .getOrCreate()
)

spark.sparkContext.setLogLevel("ERROR")
print(f" Spark {spark.version} iniciado\n")


:: loading settings :: url = jar:file:/opt/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
io.delta#delta-spark_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-9f7ab7fa-b8ae-4a61-84d1-b919d69880a2;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.5.1 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.5.1 in central
	found org.apache.kafka#kafka-clients;3.4.1 in central
	found org.lz4#lz4-java;1.8.0 in central
	found org.xerial.snappy#snappy-java;1.1.10.3 in central
	found org.slf4j#slf4j-api;2.0.7 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.4 in central
	found org.apache.hadoop#hadoop-client-api;3.3.4 in central
	found commons-logging#commons-logging;1.1.3 in central
	found com.google.code.findbugs#jsr305;3.0.0 in central
	found org.apache.commons#commons-pool2;2.11.1 in central
	found io.delta#delta-spark_

 Spark 3.5.1 iniciado



In [2]:
# ============================================================================
# 1. LECTURA DE KAFKA
# ============================================================================
print("="*80)
print("PASO 1: LECTURA DE KAFKA")
print("="*80 + "\n")

# Schema actualizado - SIN municipio, coincide con el producer
contract_schema = StructType([
    StructField("id_contrato", StringType()),
    StructField("objeto_contrato", StringType()),
    StructField("entidad", StringType()),
    StructField("departamento", StringType()),
    StructField("region", StringType()),  # SIN municipio
    StructField("codigo_unspsc", StringType()),
    StructField("descripcion_categoria", StringType()),
    StructField("valor_contrato", DoubleType()),
    StructField("duracion_dias", IntegerType()),
    StructField("fecha_firma", StringType()),
    StructField("tipo_contrato", StringType()),
    StructField("estado_contrato", StringType()),
    StructField("modalidad", StringType()),
    StructField("anno", IntegerType()),
    StructField("id_interno_sistema", StringType()),
    StructField("campo_vacio", StringType()),
    StructField("constante_1", StringType()),
    StructField("constante_2", IntegerType()),
    StructField("duplicate_id", StringType()),
    StructField("timestamp_carga", StringType())
])

print("Conectando a Kafka...")
print("Broker: kafka:29092")
print("Tópico: contratos-publicos")
print("Modo: Batch (startingOffsets=earliest)\n")

try:
    df_kafka = spark.read \
        .format("kafka") \
        .option("kafka.bootstrap.servers", "kafka:29092") \
        .option("subscribe", "contratos-publicos") \
        .option("startingOffsets", "earliest") \
        .option("failOnDataLoss", "false") \
        .load()
    
    print("✅ Conexión exitosa a Kafka\n")
    
    # Parsear JSON desde Kafka
    print("Parseando mensajes JSON...")
    df_bronze = df_kafka.select(
        from_json(col("value").cast("string"), contract_schema).alias("data")
    ).select("data.*")
    
    # Cache para evitar múltiples lecturas
    df_bronze = df_bronze.cache()
    
    # Contar mensajes
    print("Contando mensajes...")
    total_kafka = df_bronze.count()
    
    print("\n" + "="*80)
    print(f" LECTURA COMPLETADA")
    print("="*80)
    print(f" Total de mensajes leídos: {total_kafka:,}")
    print(f" Campos en el schema: {len(contract_schema.fields)}")
    print("="*80 + "\n")
    
    # Mostrar muestra de datos
    if total_kafka > 0:
        print("Muestra de los primeros 3 registros:")
        df_bronze.show(3, truncate=True)
        print()
    else:
        print(" ADVERTENCIA: No se encontraron mensajes en el tópico")
        print("Verifica que el producer haya enviado datos correctamente\n")

except Exception as e:
    print("\n" + "="*80)
    print(" ERROR EN LECTURA DE KAFKA")
    print("="*80)
    print(f"Error: {str(e)}\n")
    raise

PASO 1: LECTURA DE KAFKA

Conectando a Kafka...
Broker: kafka:29092
Tópico: contratos-publicos
Modo: Batch (startingOffsets=earliest)

✅ Conexión exitosa a Kafka

Parseando mensajes JSON...
Contando mensajes...



 LECTURA COMPLETADA
 Total de mensajes leídos: 100,946
 Campos en el schema: 20

Muestra de los primeros 3 registros:
+------------+--------------------+--------------------+------------+--------------+-------------+---------------------+--------------+-------------+-----------+--------------------+---------------+--------------------+----+------------------+-----------+-----------+-----------+------------+--------------------+
| id_contrato|     objeto_contrato|             entidad|departamento|        region|codigo_unspsc|descripcion_categoria|valor_contrato|duracion_dias|fecha_firma|       tipo_contrato|estado_contrato|           modalidad|anno|id_interno_sistema|campo_vacio|constante_1|constante_2|duplicate_id|     timestamp_carga|
+------------+--------------------+--------------------+------------+--------------+-------------+---------------------+--------------+-------------+-----------+--------------------+---------------+--------------------+----+------------------+----------

In [3]:
# ============================================================================
# 2. ELIMINAR REDUNDANTES Y PREPARAR DATOS
# ============================================================================
print("="*80)
print("PASO 2: ELIMINAR REDUNDANTES Y PREPARAR DATOS")
print("="*80 + "\n")

# Columnas redundantes a eliminar
redundant_columns = [
    "id_interno_sistema", 
    "campo_vacio", 
    "constante_1",
    "constante_2", 
    "duplicate_id", 
    "timestamp_carga"
]

print(f"📌 Eliminando {len(redundant_columns)} columnas redundantes...")
df_cleaned = df_bronze.drop(*redundant_columns)

print(f"✅ Columnas restantes: {len(df_cleaned.columns)}")
print()

# CRÍTICO: Convertir fecha_firma ANTES de hacer cualquier operación
print("🔄 Preparando campo fecha_firma...")
print("   Formato recibido: ISO timestamp (2024-01-04T00:00:00.000)")
print("   Convirtiendo a: date (2024-01-04)")

df_cleaned = df_cleaned.withColumn(
    "fecha_firma_temp",
    to_timestamp(col("fecha_firma"))  # Parsea el timestamp ISO
).withColumn(
    "fecha_firma",
    to_date(col("fecha_firma_temp"))  # Extrae solo la fecha
).drop("fecha_firma_temp")

print("✅ Fecha convertida correctamente\n")

# Liberar bronze ahora que ya no lo necesitamos
print("🧹 Liberando memoria de df_bronze...")
df_bronze.unpersist()
print("✅ Memoria liberada\n")

print("="*80)
print(f"📊 Dataset preparado: {len(df_cleaned.columns)} columnas")
print("="*80 + "\n")

PASO 2: ELIMINAR REDUNDANTES Y PREPARAR DATOS

📌 Eliminando 6 columnas redundantes...
✅ Columnas restantes: 14

🔄 Preparando campo fecha_firma...
   Formato recibido: ISO timestamp (2024-01-04T00:00:00.000)
   Convirtiendo a: date (2024-01-04)
✅ Fecha convertida correctamente

🧹 Liberando memoria de df_bronze...
✅ Memoria liberada

📊 Dataset preparado: 14 columnas



In [4]:
# ============================================================================
# CELDA 1: PREPARACIÓN Y CONTEO INICIAL
# ============================================================================
print("="*80)
print("PASO 3: LIMPIEZA - PREPARACIÓN")
print("="*80 + "\n")

print("Cacheando datos para análisis...")
df_cleaned = df_cleaned.cache()
total_cleaned = df_cleaned.count()

print(f"✅ Registros totales: {total_cleaned:,}\n")
print(f"📋 Columnas: {len(df_cleaned.columns)}")
print(f"💾 Datos cacheados en memoria\n")

PASO 3: LIMPIEZA - PREPARACIÓN

Cacheando datos para análisis...


[Stage 5:>                                                          (0 + 1) / 1]

✅ Registros totales: 100,946

📋 Columnas: 14
💾 Datos cacheados en memoria



In [5]:
# ============================================================================
# CELDA 2: ANÁLISIS DE NULOS (OPTIMIZADO)
# ============================================================================
print("="*80)
print("ANÁLISIS DE CALIDAD DE DATOS")
print("="*80 + "\n")

print("Analizando valores nulos en columnas críticas...")

# Solo analizar columnas críticas para ahorrar memoria
critical_columns = [
    "id_contrato",
    "objeto_contrato", 
    "valor_contrato",
    "fecha_firma",
    "entidad",
    "departamento",
    "duracion_dias"
]

# Análisis optimizado solo de columnas críticas
null_analysis = df_cleaned.select([
    count(when(col(c).isNull(), c)).alias(c) 
    for c in critical_columns if c in df_cleaned.columns
]).collect()[0].asDict()

print("📊 Valores nulos en columnas críticas:\n")
has_nulls = False
for col_name in critical_columns:
    if col_name in null_analysis:
        null_count = null_analysis[col_name]
        if null_count > 0:
            has_nulls = True
            pct = (null_count / total_cleaned) * 100
            print(f"   ⚠️  {col_name}: {null_count:,} ({pct:.1f}%)")

if not has_nulls:
    print("   ✅ No hay valores nulos en columnas críticas")

print()

ANÁLISIS DE CALIDAD DE DATOS

Analizando valores nulos en columnas críticas...
📊 Valores nulos en columnas críticas:

   ⚠️  fecha_firma: 695 (0.7%)
   ⚠️  duracion_dias: 50,350 (49.9%)



In [6]:
# ============================================================================
# CELDA 3: APLICAR FILTROS DE LIMPIEZA
# ============================================================================
print("="*80)
print("APLICANDO FILTROS DE CALIDAD")
print("="*80 + "\n")

print("Aplicando reglas de limpieza:")
print("  ✓ id_contrato no nulo")
print("  ✓ objeto_contrato no nulo")
print("  ✓ valor_contrato no nulo y > 0")
print("  ✓ fecha_firma no nula")
print()

# Aplicar filtros paso a paso
# NOTA: fecha_firma ya fue convertida a date en el Paso 2
df_silver = df_cleaned \
    .filter(col("id_contrato").isNotNull()) \
    .filter(col("objeto_contrato").isNotNull()) \
    .filter(col("valor_contrato").isNotNull()) \
    .filter(col("valor_contrato") > 0) \
    .filter(col("fecha_firma").isNotNull())

print("✅ Filtros aplicados correctamente\n")

APLICANDO FILTROS DE CALIDAD

Aplicando reglas de limpieza:
  ✓ id_contrato no nulo
  ✓ objeto_contrato no nulo
  ✓ valor_contrato no nulo y > 0
  ✓ fecha_firma no nula

✅ Filtros aplicados correctamente



In [7]:
# ============================================================================
# CELDA 4: CACHEAR RESULTADOS Y GENERAR REPORTE
# ============================================================================
print("="*80)
print("FINALIZANDO LIMPIEZA")
print("="*80 + "\n")

print("Cacheando datos limpios...")
df_silver = df_silver.cache()
total_silver = df_silver.count()

# Calcular estadísticas
registros_descartados = total_cleaned - total_silver
pct_retenido = (total_silver / total_cleaned) * 100 if total_cleaned > 0 else 0
pct_descartado = (registros_descartados / total_cleaned) * 100 if total_cleaned > 0 else 0

print("\n" + "="*80)
print("📊 RESUMEN DE LIMPIEZA")
print("="*80)
print(f"  Registros iniciales:    {total_cleaned:,}")
print(f"  Registros finales:      {total_silver:,} ({pct_retenido:.1f}%)")
print(f"  Registros descartados:  {registros_descartados:,} ({pct_descartado:.1f}%)")
print("="*80 + "\n")

# Liberar memoria del DataFrame anterior
print("Liberando memoria del cache anterior...")
df_cleaned.unpersist()
print("✅ Limpieza completada\n")

FINALIZANDO LIMPIEZA

Cacheando datos limpios...


[Stage 12:>                                                         (0 + 1) / 1]


📊 RESUMEN DE LIMPIEZA
  Registros iniciales:    100,946
  Registros finales:      99,706 (98.8%)
  Registros descartados:  1,240 (1.2%)

Liberando memoria del cache anterior...
✅ Limpieza completada



In [8]:
# ============================================================================
# 4. ESTADÍSTICAS
# ============================================================================

print("="*80)
print("PASO 4: ESTADÍSTICAS")
print("="*80 + "\n")

print("📊 Por región:")
df_silver.groupBy("codigo_unspsc").count().orderBy(desc("count")).show(5)

print("\n📊 Top 5 entidades:")
df_silver.groupBy("entidad").count().orderBy(desc("count")).show(5, truncate=False)


PASO 4: ESTADÍSTICAS

📊 Por región:
+-------------+-----+
|codigo_unspsc|count|
+-------------+-----+
|             |50058|
|  V1.80111600|11418|
|  V1.80111701| 4341|
|  V1.85101600| 2605|
|  V1.80111620| 2202|
+-------------+-----+
only showing top 5 rows


📊 Top 5 entidades:
+-------------------------------------------------+-----+
|entidad                                          |count|
+-------------------------------------------------+-----+
|MUNICIPIO DE SOACHA.                             |6354 |
|ALCALDÍA MUNICIPAL COTA                          |3987 |
|ESE MUNICIPAL DE SOACHA JULIO CESAR PEÑALOZA*    |3822 |
|CUNDINAMARCA-ALCALDIA MUNICIPIO MOSQUERA         |3764 |
|empresa social del estado región de salud soacha.|3152 |
+-------------------------------------------------+-----+
only showing top 5 rows



In [10]:
# 1. Entidades
print("📊 Top 5 entidades:")
df_silver.groupBy("entidad").count().orderBy(desc("count")).show(5, truncate=False)

# 2. Departamentos
print("\n📊 Top 5 departamentos:")
df_silver.groupBy("departamento").count().orderBy(desc("count")).show(5, truncate=False)

# 4. Región
print("\n📊 Distribución por región:")
df_silver.groupBy("region").count().orderBy(desc("count")).show(truncate=False)

# 5. Código UNSPSC
print("\n📊 Top 10 códigos UNSPSC:")
df_silver.groupBy("codigo_unspsc").count().orderBy(desc("count")).show(10, truncate=False)

# 6. Categoría UNSPSC
print("\n📊 Top 10 categorías UNSPSC:")
df_silver.groupBy("descripcion_categoria").count().orderBy(desc("count")).show(10, truncate=False)

# 7. Tipo de contrato
print("\n📊 Distribución por tipo de contrato:")
df_silver.groupBy("tipo_contrato").count().orderBy(desc("count")).show(truncate=False)

# 8. Estado del contrato
print("\n📊 Distribución de estado del contrato:")
df_silver.groupBy("estado_contrato").count().orderBy(desc("count")).show(truncate=False)

# 9. Modalidad de contratación
print("\n📊 Top 10 modalidades:")
df_silver.groupBy("modalidad").count().orderBy(desc("count")).show(10, truncate=False)


📊 Top 5 entidades:
+-------------------------------------------------+-----+
|entidad                                          |count|
+-------------------------------------------------+-----+
|MUNICIPIO DE SOACHA.                             |6354 |
|ALCALDÍA MUNICIPAL COTA                          |3987 |
|ESE MUNICIPAL DE SOACHA JULIO CESAR PEÑALOZA*    |3822 |
|CUNDINAMARCA-ALCALDIA MUNICIPIO MOSQUERA         |3764 |
|empresa social del estado región de salud soacha.|3152 |
+-------------------------------------------------+-----+
only showing top 5 rows


📊 Top 5 departamentos:
+------------+-----+
|departamento|count|
+------------+-----+
|Cundinamarca|99706|
+------------+-----+


📊 Distribución por región:
+--------------+-----+
|region        |count|
+--------------+-----+
|Centro-Oriente|99706|
+--------------+-----+


📊 Top 10 códigos UNSPSC:
+-------------+-----+
|codigo_unspsc|count|
+-------------+-----+
|             |50058|
|V1.80111600  |11418|
|V1.80111701  |4341 |
|V

In [11]:
from pyspark.sql.functions import min, max, avg, stddev, expr

# 10. Valor del contrato
print("\n💰 Estadísticas de valor_contrato:")
df_silver.select(
    min("valor_contrato").alias("min"),
    max("valor_contrato").alias("max"),
    avg("valor_contrato").alias("mean"),
    stddev("valor_contrato").alias("std")
).show()

# Percentiles
print("\n💰 Percentiles de valor_contrato:")
df_silver.approxQuantile("valor_contrato", [0.01, 0.25, 0.5, 0.75, 0.99], 0.01)

# 11. Duración en días
print("\n⏱️ Estadísticas de duracion_dias:")
df_silver.select(
    min("duracion_dias").alias("min"),
    max("duracion_dias").alias("max"),
    avg("duracion_dias").alias("mean"),
    stddev("duracion_dias").alias("std")
).show()

print("\n⏱️ Percentiles de duracion_dias:")
df_silver.approxQuantile("duracion_dias", [0.01, 0.25, 0.5, 0.75, 0.99], 0.01)



💰 Estadísticas de valor_contrato:
+---+----------------+------------------+--------------------+
|min|             max|              mean|                 std|
+---+----------------+------------------+--------------------+
|1.0|1.50838540149E11|9.96561478195896E7|1.1539535981771367E9|
+---+----------------+------------------+--------------------+


💰 Percentiles de valor_contrato:

⏱️ Estadísticas de duracion_dias:
+---+----+-----------------+------------------+
|min| max|             mean|               std|
+---+----+-----------------+------------------+
|  0|4297|83.20190545249461|101.88598302532633|
+---+----+-----------------+------------------+


⏱️ Percentiles de duracion_dias:


[0.0, 6.0, 40.0, 126.0, 4297.0]

In [10]:
print("\n📅 Top años:")
df_silver.groupBy("anno").count().orderBy(desc("anno")).show(10, truncate=False)

print("\n📅 Contratos por año:")
df_silver.groupBy("anno").count().orderBy(desc("count")).show(10, truncate=False)

print("\n📅 Top fechas de firma:")
df_silver.groupBy("fecha_firma").count().orderBy(desc("count")).show(10, truncate=False)



📅 Top años:
+----+-----+
|anno|count|
+----+-----+
|2024|50058|
+----+-----+


📅 Contratos por año:
+----+-----+
|anno|count|
+----+-----+
|2024|50058|
+----+-----+


📅 Top fechas de firma:
+-----------+-----+
|fecha_firma|count|
+-----------+-----+
|2024-03-01 |457  |
|2024-02-01 |450  |
|2024-03-22 |436  |
|2024-02-02 |425  |
|2024-01-31 |415  |
|2024-02-16 |398  |
|2024-02-09 |398  |
|2024-02-06 |363  |
|2024-09-16 |343  |
|2024-02-08 |343  |
+-----------+-----+
only showing top 10 rows



In [12]:

# --------------------------------------------
# TEXTO / OBJETO DEL CONTRATO
# --------------------------------------------

print("\n📝 Longitud promedio del objeto del contrato:")
df_silver.select(avg(expr("length(objeto_contrato)"))).show()


📝 Longitud promedio del objeto del contrato:
+----------------------------+
|avg(length(objeto_contrato))|
+----------------------------+
|          224.29975126873006|
+----------------------------+



In [13]:
# ============================================================================
# 5. GUARDAR EN DELTA LAKE
# ============================================================================

print("="*80)
print("PASO 5: GUARDAR EN DELTA LAKE")
print("="*80 + "\n")

DELTA_PATH = "/app/notebooks/delta_lake/silver_contracts"

print(f"💾 Guardando en: {DELTA_PATH}")

df_silver.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .save(DELTA_PATH)

print("✅ Guardado exitosamente\n")

# ⚠️ LIBERAR todo
df_silver.unpersist()
spark.catalog.clearCache()


PASO 5: GUARDAR EN DELTA LAKE

💾 Guardando en: /app/notebooks/delta_lake/silver_contracts


✅ Guardado exitosamente



In [10]:
print(f"Spark version: {spark.version}")

Spark version: 3.5.1


In [14]:
# ============================================================================
# 6. VERIFICACIÓN
# ============================================================================

print("="*80)
print("VERIFICACIÓN FINAL")
print("="*80 + "\n")

df_verify = spark.read.format("delta").load(DELTA_PATH)
print(f"✅ Registros verificados: {df_verify.count():,}")

df_verify.select("id_contrato", "entidad", "valor_contrato", "fecha_firma") \
    .show(5, truncate=False)

print("\n🎯 Fase 2 completada. Siguiente: Fase 3 - Embeddings\n")

VERIFICACIÓN FINAL

✅ Registros verificados: 99,706
+------------+----------------------------------------------------+--------------+-----------+
|id_contrato |entidad                                             |valor_contrato|fecha_firma|
+------------+----------------------------------------------------+--------------+-----------+
|CPS-045-2024|empresa social del estado región de salud soacha.   |7.8624E7      |2024-01-01 |
|CPS 018-2024|E.S.E HOSPITAL NUESTRA SEÑORA DEL CARMEN DEL COLEGIO|1.07844E7     |2024-01-01 |
|CPS 012-2024|E.S.E HOSPITAL NUESTRA SEÑORA DEL CARMEN DEL COLEGIO|1.07844E7     |2024-01-01 |
|024-2024    |ESE HOSPITAL SALAZAR DE VILLETA                     |9363575.0     |2024-01-01 |
|CPS-060-2024|empresa social del estado región de salud soacha.   |8.3279308E7   |2024-01-01 |
+------------+----------------------------------------------------+--------------+-----------+
only showing top 5 rows


🎯 Fase 2 completada. Siguiente: Fase 3 - Embeddings



In [2]:
# ============================================================================
# 1. CARGAR DATOS DESDE SILVER
# ============================================================================

print("\n" + "="*80)
print("PASO 1: CARGAR DATOS DESDE SILVER")
print("="*80 + "\n")

SILVER_PATH = "/app/notebooks/delta_lake/silver_contracts"

print(f"📖 Cargando datos desde: {SILVER_PATH}")

df_silver = spark.read.format("delta").load(SILVER_PATH)
df_silver = df_silver.cache()

total_records = df_silver.count()
print(f"✅ Registros cargados: {total_records:,}\n")

print("📋 Esquema de datos:")
df_silver.printSchema()

print("\n📊 Muestra de datos:")
df_silver.select(
    "id_contrato", "objeto_contrato", "entidad", 
    "codigo_unspsc", "valor_contrato", "duracion_dias"
).show(3, truncate=False)





PASO 1: CARGAR DATOS DESDE SILVER

📖 Cargando datos desde: /app/notebooks/delta_lake/silver_contracts


[Stage 9:=============================>                             (1 + 1) / 2]

✅ Registros cargados: 50,058

📋 Esquema de datos:
root
 |-- id_contrato: string (nullable = true)
 |-- objeto_contrato: string (nullable = true)
 |-- entidad: string (nullable = true)
 |-- departamento: string (nullable = true)
 |-- municipio: string (nullable = true)
 |-- region: string (nullable = true)
 |-- codigo_unspsc: string (nullable = true)
 |-- descripcion_categoria: string (nullable = true)
 |-- valor_contrato: double (nullable = true)
 |-- duracion_dias: integer (nullable = true)
 |-- fecha_firma: date (nullable = true)
 |-- tipo_contrato: string (nullable = true)
 |-- estado_contrato: string (nullable = true)
 |-- modalidad: string (nullable = true)
 |-- anno: integer (nullable = true)


📊 Muestra de datos:
+------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [3]:
print("\n" + "="*80)
print("PASO 2: LIMPIEZA Y PREPARACIÓN DE TEXTO (NORMALIZADO)")
print("="*80 + "\n")

print("🧹 Limpiando columna 'objeto_contrato' y eliminando tildes...")

# Definimos los caracteres a reemplazar
# Nota: Incluí la 'ñ' -> 'n' y la 'ü' -> 'u'. 
# Si quieres CONSERVAR la ñ, quítala de estas cadenas.
src_chars = "áéíóúüñ"
dst_chars = "aeiouun"

df_prepared = df_silver.withColumn(
    "objeto_limpio",
    trim(                                           # 4. Quitar espacios al inicio/final
        regexp_replace(                             # 3. Colapsar espacios múltiples
            regexp_replace(                         # 2. Eliminar caracteres especiales
                translate(                          # 1. Reemplazar tildes
                    lower(col("objeto_contrato")),  # 0. Convertir a minúsculas
                    src_chars, 
                    dst_chars
                ),
                "[^a-z0-9\\s]", " " # Solo deja letras a-z (sin tildes), números y espacios
            ),
            "\\s+", " "
        )
    )
)

# Filtrar textos muy cortos
df_prepared = df_prepared.filter(length(col("objeto_limpio")) >= 10)

print(f" Registros después de limpieza: {df_prepared.count():,}")

print("\n Ejemplo de texto limpio (Sin tildes):")
df_prepared.select("objeto_contrato", "objeto_limpio").show(5, truncate=50)


PASO 2: LIMPIEZA Y PREPARACIÓN DE TEXTO (NORMALIZADO)

🧹 Limpiando columna 'objeto_contrato' y eliminando tildes...


ERROR:root:Exception while sending command.                         (1 + 1) / 2]
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/site-packages/py4j/clientserver.py", line 516, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.11/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/site-packages/py4j/clientserver.py", line 539, in send_command
    raise Py4JNetworkError(
py4j.protocol.Py4JNetworkError: Error while sending or receiving
ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/site-packages/py4j/clientserver.py", line 516, in send_comman

Py4JError: An error occurred while calling o72.count

In [4]:
# ============================================================================
# PASO 3.1: DEFINIR STOPWORDS Y TOKENIZAR
# ============================================================================

print("🔤 PASO 3.1: Tokenización y Stopwords")
print("-" * 80 + "\n")

# Stopwords especializados para contratos
stopwords_es = [
    # Básicos español
    "el", "la", "de", "que", "y", "a", "en", "un", "ser", "se", "no",
    "por", "con", "su", "para", "como", "estar", "tener", "le", "lo",
    "pero", "hacer", "o", "este", "otro", "ese", "si", "ya", "ver",
    "dar", "muy", "sin", "sobre", "también", "hasta", "año", "entre",
    "del", "al", "los", "las", "uno", "una", "unos", "unas",
    # Específicos de contratos públicos
    "contrato", "contratos", "objeto", "prestacion", "prestación",
    "servicio", "servicios", "suministro", "ejecucion", "ejecución",
    "acuerdo", "establecido", "pliego", "condiciones", "especificaciones",
    "entidad", "contratante", "contratista", "plazo", "termino"
]

print(f"   • Stopwords definidos: {len(stopwords_es)} (básicos + contratos)")

# Tokenización
tokenizer = Tokenizer(inputCol="objeto_limpio", outputCol="palabras")
df_tokenized = tokenizer.transform(df_prepared)

# Remover stopwords
remover = StopWordsRemover(
    inputCol="palabras", 
    outputCol="palabras_sin_stopwords",
    stopWords=stopwords_es
)
df_filtered = remover.transform(df_tokenized)

print("   ✅ Tokenización y stopwords completados\n")

print("📋 Ejemplo:")
df_filtered.select("objeto_limpio", "palabras_sin_stopwords").show(2, truncate=70)


🔤 PASO 3.1: Tokenización y Stopwords
--------------------------------------------------------------------------------

   • Stopwords definidos: 65 (básicos + contratos)
   ✅ Tokenización y stopwords completados

📋 Ejemplo:
+----------------------------------------------------------------------+----------------------------------------------------------------------+
|                                                         objeto_limpio|                                                palabras_sin_stopwords|
+----------------------------------------------------------------------+----------------------------------------------------------------------+
|prestacion de servicios como medico general con experiencia en coor...|[medico, general, experiencia, coordinacion, hospitalizacion, urgen...|
|prestar apoyo al proceso asistencial en el area de farmacia tecnica...|[prestar, apoyo, proceso, asistencial, area, farmacia, tecnica, e, ...|
+-------------------------------------------------------

In [5]:
# ============================================================================
# PASO 3.2: LIMPIEZA Y FILTRADO
# ============================================================================

print("\n🔍 PASO 3.2: Limpieza y Filtrado")
print("-" * 80 + "\n")

# Filtrar palabras muy cortas (ruido)
def clean_words(words):
    """Mantener solo palabras >= 3 caracteres"""
    if not words:
        return []
    return [w for w in words if len(w) >= 3]

clean_udf = udf(clean_words, ArrayType(StringType()))

df_filtered = df_filtered.withColumn(
    "palabras_filtradas",
    clean_udf(col("palabras_sin_stopwords"))
)

# Eliminar documentos vacíos
df_filtered = df_filtered.filter(size(col("palabras_filtradas")) > 0)

print("   • Filtro aplicado: palabras >= 3 caracteres")
print("   • Documentos vacíos eliminados")
print("   ✅ Limpieza completada\n")




🔍 PASO 3.2: Limpieza y Filtrado
--------------------------------------------------------------------------------

   • Filtro aplicado: palabras >= 3 caracteres
   • Documentos vacíos eliminados
   ✅ Limpieza completada



In [7]:
# ============================================================================
# 4. GENERACIÓN DE EMBEDDINGS CON WORD2VEC
# ============================================================================

print("\n" + "="*80)
print("PASO 4: GENERACIÓN DE EMBEDDINGS CON WORD2VEC")
print("="*80 + "\n")

print("🔢 Entrenando modelo Word2Vec...")
print("   - Vector size: 100 dimensiones")
print("   - Min word count: 2 (palabras que aparecen al menos 2 veces)")
print("   - Iterations: 10\n")

# Configurar Word2Vec
word2vec = Word2Vec(
    vectorSize=100,
    minCount=2,
    maxIter=10,
    seed=42,
    inputCol="palabras_filtradas",
    outputCol="embedding_raw"
)

# Entrenar Word2Vec
print("⏳ Entrenando (esto puede tardar 1-2 minutos)...")
word2vec_model = word2vec.fit(df_filtered)

# Aplicar el modelo
df_embeddings = word2vec_model.transform(df_filtered)

print("✅ Word2Vec entrenado y aplicado")

# Estadísticas del vocabulario
vocab_size = len(word2vec_model.getVectors().collect())
print(f"\n📊 Tamaño del vocabulario: {vocab_size:,} palabras únicas")

print("\n📋 Ejemplo de embeddings (primeros 10 valores):")

# SOLUCIÓN: Crear UDF para convertir Vector a Array
from pyspark.sql.functions import udf
from pyspark.sql.types import ArrayType, DoubleType
from pyspark.ml.linalg import VectorUDT, Vectors

# UDF para convertir Vector de Spark ML a Array
def vector_to_array(vector):
    return vector.toArray().tolist() if vector is not None else None

vector_to_array_udf = udf(vector_to_array, ArrayType(DoubleType()))

# Convertir el embedding a array y mostrar primeros 10 valores
df_embeddings.select(
    "id_contrato",
    slice(vector_to_array_udf(col("embedding_raw")), 1, 10).alias("embedding_muestra")
).show(2, truncate=False)

# Opcional: Ver el tamaño completo del embedding
print("\n📏 Verificando dimensión del embedding:")
df_embeddings.select(
    "id_contrato",
    size(vector_to_array_udf(col("embedding_raw"))).alias("dimension_embedding")
).show(5)


PASO 4: GENERACIÓN DE EMBEDDINGS CON WORD2VEC



ConnectionRefusedError: [Errno 111] Connection refused